Introduction: Introduce the data we use. Summarize the problems with the data sets and our approach to solve those problem. State the question that we are trying to answer (to predict if a provider is fraud)
Import libraries, packages, etc. and load the data, with some general info of the data (head, info, describe etc)
Check if some column/row contains no valuable info to drop them (missing value summary chart)
Deal with LEIE file to get the list of fraud NPI (name/address match if possible), call this df_label
Join df_label with data set B, D and DME on NPI and aggregate
Remove unneccessary columns, maybe we can try PCA a little bit?
Top cities, states, general, specialty etc. involve in fraud
Histogram on each features to see if there is any difference between non-fraud and fraud?
Features selection (I'm not sure how we should proceed?)
Plot relationship between selected features and potential_fraud
Conclusion/comments

In [ ]:
#import libraries
import os
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#import data
dat_B = pd.read_csv(os.path.join(folder_path, 'CMS', 'formB_MUP_PHY_R21_P04_V10_D19_Prov_Svc-001.csv'), encoding = "ISO-8859-1")
dat_D = pd.read_csv(os.path.join(folder_path, 'CMS', 'formD_MUP_DPR_RY21_P04_V10_DY19_NPIBN_1-002.csv'), encoding = "ISO-8859-1")
dat_dme = pd.read_csv(os.path.join(folder_path, 'CMS', 'mup_dme_ry21_p05_v10_dy19_prvhpr_0.csv'), encoding = "ISO-8859-1")
label_df = pd.read_csv(os.path.join(folder_path, 'LEIE', 'UPDATED.csv'))

In [ ]:
#Summary of missing value
dat_B.isna().sum()
dat_D.isna().sum()
dat_dme.isna().sum()
label_df.isna().sum()

In [ ]:
#Aggregate data
dat_B_agg = dat_B.groupby(['Rndrng_NPI','Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_Cntry']).agg(
                                                   {'Tot_Srvcs':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                    'Tot_Benes':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                    'Tot_Bene_Day_Srvcs':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                    'Avg_Sbmtd_Chrg':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                    'Avg_Mdcr_Pymt_Amt': ['sum', 'mean', 'median', np.std, 'min', 'max']
                                                 })
dat_B_agg.head()


In [ ]:
dat_D_agg = dat_D.groupby(['Prscrbr_NPI','Prscrbr_Type']).agg(
                                                  {'Tot_Benes':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                   'Tot_Clms':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                   'Tot_30day_Fills':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                   'Tot_Day_Suply':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                   'Tot_Drug_Cst': ['sum', 'mean', 'median', np.std, 'min', 'max']
                                                  })
dat_D_agg.head()

In [ ]:
dat_dme_agg = dat_dme.groupby(['Rfrg_NPI',]).agg({'Tot_Suplrs':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                       'Tot_Suplr_Benes':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                       'Tot_Suplr_Clms':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                       'Tot_Suplr_Srvcs':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                       'Avg_Suplr_Mdcr_Pymt_Amt':['sum', 'mean', 'median', np.std, 'min', 'max'],
                                                       'Avg_Suplr_Mdcr_Stdzd_Amt': ['sum', 'mean', 'median', np.std, 'min', 'max']
                                                 })
                                                                                    
dat_dme_agg.head()

In [ ]:
#merge data based on NPI
merged_d = dat_dme_agg.reset_index().merge(dat_D_agg, how='inner', left_on=['Rfrg_NPI'], right_on=['Prscrbr_NPI'])
merged_d = merged_d.merge(dat_B_agg, how='inner', left_on='Rfrg_NPI', right_on='Rndrng_NPI')

In [ ]:
#processing the label data
df_label_fraud = label_df[label_df['NPI'] != 0] #get all rows that have nonzero NPI
df_label_fraud = df_label_fraud[~df_label_fraud.index.duplicated(keep='first')].reset_index('NPI') #remove duplicated NPI
pd.Series(df_label_fraud.NPI).is_unique #check if the NPI is indeed unique

In [ ]:
#Top 10 States involving in Fraud
sns.set(rc={'figure.figsize':(12,8)},style='white')

ax=sns.countplot(x='STATE',data=df_label_fraud
              ,order=df_label_fraud.STATE.value_counts().iloc[:10].index)

plt.title('Top-10 States involved in Healthcare Fraud')
    
plt.show()

In [ ]:
#Top 10 Cities involving in Fraud
df_label_fraud['city'] = df_label_fraud['CITY'] + ', ' + df_label_fraud['STATE'] #we add state to city to avoid problem of same city name for different states
print(df_label_fraud.city.value_counts())
#sns.set(rc={'figure.figsize':(12,8)},style='white')
ax=sns.countplot(x='city',data=df_label_fraud
             ,order=df_label_fraud.city.value_counts().iloc[:10].index)

plt.title('Top-10 Cities involved in Healthcare Fraud')
    
plt.show()

We see that Miami, FL is the city with most healthcare fraud although Florida is only at 3rd place in the list of states with most fraud.

In [ ]:
#Top 10 generals involving in Fraud
sns.set(rc={'figure.figsize':(12,8)},style='white')

ax=sns.countplot(x='GENERAL',data=df_label_fraud
              ,order=df_label_fraud.GENERAL.value_counts().iloc[:10].index)

plt.title('Top-10 Generals involved in Healthcare Fraud')
    
plt.show()

The number of fraud in IND- LIC HC SERV PRO and PHYSICIAN (MD, DO) are outstanding compared to others.

In [ ]:
# create a dataframe, named df_label, with first column is NPI and second column is Potential_Fraud 
label = df_label_fraud['NPI']
df_label = {"NPI":[],"Potential_Fraud":[]}
for i in label:
    df_label['NPI'].append(i) 
    df_label['Potential_Fraud'].append('Yes')
df_label = pd.DataFrame(df_label)

In [ ]:
#Checking variance of average amount charged based on type of service
dat_B.groupby('HCPCS_Cd').agg({'Avg_Sbmtd_Chrg':np.mean}).sort_values('Avg_Sbmtd_Chrg')
df_grouped = dat_B.groupby([, 'Rndrng_NPI']).agg({'Avg_Sbmtd_Chrg':np.mean}).sort_values('Avg_Sbmtd_Chrg')
df_test = dat_B.groupby(['HCPCS_Cd']).agg({'Avg_Sbmtd_Chrg': lambda x: np.std(x)})
import matplotlib.pyplot as plt
from numpy.ma.core import std
plt.hist(df_test['Avg_Sbmtd_Chrg'], bins = 50)
plt.show()
df_test_greater_than_50 = df_test[df_test['Avg_Sbmtd_Chrg'] > np.mean(df_test['Avg_Sbmtd_Chrg'])]
plt.hist(df_test_greater_than_50['Avg_Sbmtd_Chrg'])
plt.show()
df_test_greater_than_50.sort_values('Avg_Sbmtd_Chrg')

In [ ]:
leie = label_df
#mandatory minimum penalty based on offense category
minimum_exclusion_periods = { 
    '1128a1':5, #Conviction of program-related crimes. Minimum Period: 5 years
    '1128a2':5, #Conviction relating to patient abuse or neglect. Minimum Period: 5 years
    '1128a3':5, #Felony conviction relating to health care fraud. Minimum Period: 5 years
    '1128b4':np.nan, #License revocation, suspension, or surrender. Minimum Period: Period imposed by the state licensing authority.    
    '1128b7':np.nan, #Fraud, kickbacks, and other prohibited activities. Minimum Period: None
    '1128c3gi':10, #Conviction of second mandatory exclusion offense. Minimum Period: 10 years
    '1128c3gii':100 #Conviction of third or more mandatory exclusion offenses. Permanent Exclusion
}
exclusion_codes = ['1128a1', '1128a2', '1128a3', '1128b4', '1128b7', '1128c3gi', '1128c3gii']
filtered_leie = leie[leie.EXCLTYPE.isin(exclusion_codes)]
filtered_leie.shape
filtered_leie.columns

In [ ]:
#convert to time
filtered_leie['EXCLDATE'] = pd.to_datetime(filtered_leie.EXCLDATE, format='%Y%m%d') #most important value
filtered_leie['WAIVERDATE'] = pd.to_datetime(filtered_leie.WAIVERDATE, format='%Y%m%d', errors='coerce')
filtered_leie['REINDATE'] = pd.to_datetime(filtered_leie.REINDATE, format='%Y%m%d', errors='coerce')

In [ ]:
print("Unique `WAIVERDATE` values: ", len(filtered_leie['WAIVERDATE'].unique()))
print("Unique `REINDATE` values: ", len(filtered_leie['REINDATE'].unique()))

In [ ]:
#mapping the penalty to the minimum exclusions period, placing minimum exclusions on their own col

filtered_leie['MIN_EXCLUSION_PERIOD'] = filtered_leie['EXCLTYPE'].map(minimum_exclusion_periods)
filtered_leie.columns

In [ ]:
filtered_leie['END_EXCLDATE'] = filtered_leie.EXCLDATE.dt.year + filtered_leie.MIN_EXCLUSION_PERIOD
filtered_leie.columns

In [ ]:
def calculate_exclusion_end(exclusion_date, end_year):
    month = exclusion_date.month
    

    if month > 6:
        return end_year + 1
    else:
        return end_year

In [ ]:
#applying the function that calculates end dates

filtered_leie['END_EXCLDATE'] = filtered_leie[["EXCLDATE","END_EXCLDATE"]].apply(lambda x: calculate_exclusion_end(*x), axis=1)
filtered_leie.columns
filtered_leie = filtered_leie[['NPI','ZIP','EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE',
       'MIN_EXCLUSION_PERIOD', 'END_EXCLDATE']]
filtered_leie
filtered_leie = filtered_leie[filtered_leie.NPI != 0]
labeled_data = pd.merge(merged_d, filtered_leie, on='NPI',how='left') #add a column to the data which contain the fraud information.
labeled_data['TARGET'] = '0'
labeled_data['START_EXCLDATE'] = labeled_data['EXCLDATE'].dt.year
labeled_data['DATA_YEAR'] = 2019


In [ ]:
def make_labels(start_exc, data_yr):
    if start_exc > data_yr:
        return 'FRAUD'
    else:
        return 'NOT_FRAUD'
        

In [ ]:
#applying the fraud labeling function to datasets
labeled_data['TARGET'] = labeled_data[["START_EXCLDATE","DATA_YEAR"]].apply(lambda x: make_labels(*x), axis=1)

In [ ]:
#drop unneeded columns
columns_to_drop = [ 'EXCLTYPE','EXCLDATE','REINDATE','WAIVERDATE',
                   'WVRSTATE','MIN_EXCLUSION_PERIOD','END_EXCLDATE',
                   'START_EXCLDATE', 'DATA_YEAR','ZIP']

labeled_data.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
#one hot encoding
partb_category_columns = ['provider_type', 'nppes_provider_gender']
partd_category_columns = ['specialty_description']
dmepos_category_columns = ['REFERRING_PROVIDER_GENDER', 'REFERRING_PROVIDER_TYPE']
combined_category_columns = ['provider_type', 'nppes_provider_gender']

In [ ]:
#features selection
#create heatmap on data set B
sns.heatmap(dat_B.corr())
plt.show()

In [ ]:
#We drop columns with high correlation since it's not helpful for our model
to_drop_B = ['Tot_Bene_Day_Srvcs', 'Avg_Mdcr_Alowd_Amt', 'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt']
dat_B = dat_B.drop(labels = to_drop_B, axis=1)

In [ ]:
#create heatmap on data set D
sns.heatmap(dat_D.corr())
plt.show()

In [ ]:
#We drop columns with high correlation since it's not helpful for our model
to_drop_D = ['Prscrbr_Last_Org_Name', 'Prscrbr_First_Name', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills', 'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Tot_Benes']
dat_D = dat_D.drop(labels = to_drop_D, axis=1)

In [ ]:
#create heat map on DME
sns.heatmap(dat_dme.corr())
plt.show()

In [ ]:
#We drop columns with high correlation
to_drop_dme = ['Avg_Suplr_Mdcr_Pymt_Amt', 'Avg_Suplr_Mdcr_Stdzd_Amt']
dat_dme = dat_dme.drop(labels = to_drop_dme, axis=1)